In [49]:
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [54]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = Sequential([
      Dense(32, activation="relu"),
      Dense(16, activation="relu"),
      Dense(8, activation="relu")])

    self.decoder = Sequential([
      Dense(16, activation="relu"),
      Dense(32, activation="relu"),
     Dense(4, activation="relu")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [28]:
df = pd.read_csv('../datasets/auto_encoder/bank_transactions_data_2.csv')

In [55]:
autoencoder.compile(optimizer='adam', loss='mae',run_eagerly=True)

In [16]:
df.columns

Index(['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionDate',
       'TransactionType', 'Location', 'DeviceID', 'IP Address', 'MerchantID',
       'Channel', 'CustomerAge', 'CustomerOccupation', 'TransactionDuration',
       'LoginAttempts', 'AccountBalance', 'PreviousTransactionDate'],
      dtype='object')

In [17]:
df.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39


In [56]:
df = df[['TransactionAmount','TransactionDuration',
'LoginAttempts','AccountBalance']]

In [29]:
df = df[['TransactionAmount', 'TransactionDate',
'IP Address','TransactionDuration',
'LoginAttempts','AccountBalance',
'PreviousTransactionDate']]

In [ ]:
# df['IP Address'] = df['IP Address'].apply(lambda x: int(x.replace('.','')))
# df['TransactionDate'] = df['TransactionDate'].apply(lambda x: int(x.replace('-','').replace(' ','').replace(':','')))
# df['PreviousTransactionDate'] = df['PreviousTransactionDate'].apply(lambda x: int(x.replace('-','').replace(' ','').replace(':','')))

In [31]:
df.head()

,TransactionAmount,TransactionDate,IP Address,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,14.09,20230411162914,16219821892,81,1,5112.21,20241104080808
1,376.24,20230627164419,13149614,141,1,13758.91,20241104080935
2,126.29,20230710181608,21597143157,56,1,1122.35,20241104080704
3,184.50,20230505163211,20013225150,25,1,8569.06,20241104080906
4,13.45,20231016175124,651643100,198,1,7429.40,20241104080639


In [57]:
df_train,df_test = train_test_split(df,test_size=0.2,random_state=0)

In [58]:
X = df_train.values

In [59]:
X_test = df_test.values

In [60]:
history = autoencoder.fit(X, X,
          epochs=20, 
          batch_size=10,
          validation_data=(X_test,X_test),
          shuffle=True)

Epoch 1/20
201/201 [==============================] - 11s 52ms/step - loss: 347.7278 - val_loss: 73.1707
Epoch 2/20
201/201 [==============================] - 10s 51ms/step - loss: 40.0574 - val_loss: 44.9603
Epoch 3/20
201/201 [==============================] - 10s 52ms/step - loss: 31.2457 - val_loss: 25.7555
Epoch 4/20
201/201 [==============================] - 12s 59ms/step - loss: 28.5597 - val_loss: 30.8337
Epoch 5/20
201/201 [==============================] - 11s 56ms/step - loss: 25.9045 - val_loss: 25.7298
Epoch 6/20
201/201 [==============================] - 11s 53ms/step - loss: 22.2577 - val_loss: 29.6366
Epoch 7/20
201/201 [==============================] - 11s 55ms/step - loss: 16.6195 - val_loss: 7.8994
Epoch 8/20
201/201 [==============================] - 11s 55ms/step - loss: 12.5492 - val_loss: 12.1315
Epoch 9/20
201/201 [==============================] - 10s 51ms/step - loss: 14.5560 - val_loss: 23.2420
Epoch 10/20
201/201 [==============================] - 10s 52ms/

In [61]:
reconstruction = autoencoder.predict(X)
reconstruction_error = np.mean(np.abs(X - reconstruction), axis=1)
# Calcular o limiar de erro
limiar_erro = np.mean(reconstruction_error) + 2 * np.std(reconstruction_error)

63/63 [==============================] - 1s 10ms/step


In [67]:
df_train['label'] = reconstruction_error > limiar_erro

In [70]:
len(df)

2512

In [68]:
df_train.loc[df_train['label'] == True]

,TransactionAmount,TransactionDuration,LoginAttempts,AccountBalance,label
868,83.39,158,1,13182.11,True
1726,85.73,266,1,14928.35,True
971,194.37,161,1,14395.88,True
1894,447.83,88,1,13273.08,True
1886,72.50,30,1,12723.62,True
...,...,...,...,...,...
2146,97.63,127,1,12467.31,True
1167,838.57,273,1,14576.47,True
1641,50.52,293,1,12815.24,True
2076,325.03,106,1,13521.76,True


In [73]:
reconstruction

array([[ 483.40167 ,   57.212288,    0.      , 5862.2803  ],
       [ 467.25104 ,  195.80338 ,    0.      , 7070.905   ],
       [   0.      ,   72.59162 ,    0.      , 6990.5776  ],
       ...,
       [ 467.04727 ,   59.577713,    0.      , 2154.0254  ],
       [   0.      ,  192.85632 ,    0.      , 7600.753   ],
       [  74.35798 ,  121.34133 ,    0.      , 1594.8394  ]],
      dtype=float32)